In [2]:
import tweepy
import pandas as pd
import json
import time
from datetime import datetime

In [2]:
API_KEY = "0klZ6A3ashptZfBZpEdxIZtUn"
API_SECRET = "RzDtafJJX9fO9zjykg15iMNghSMd1furq41Rpn5vfJc3M5OaQw"
ACCESS_TOKEN = "781767392607088640-BSKrUyweBAnmAbdg1EgmxVh09zmI46y"
ACCESS_TOKEN_SECRET = "Sg8wzQeocs7iQCZOI8QUv3bs5wOZo20MOXSgGvdwlIfiS"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

#Make call on home timeline, print each tweets text
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

#INFOGRAPHIC: In the UK, over 1.6 TRILLION tampons, pads and liners are flushed down toilets every year:… https://t.co/6qkHVV6Bvr
How do you take your tea? Milk, sugar, plastic?

Did you know even apart from the packaging many tea bags contain p… https://t.co/VdXPIRYNUr
RT @ACTAIglobal: Excited to speak w Oscar-winning director of #TheCove documentary &amp; @RacingXtinction on Wed, 7/8, at 10am PT.  We'll discu…
DID YOU KNOW? Whale sharks reach the size of a school bus! Learn more in our Marine Life Encyclopedia:… https://t.co/fOWZceBXMw
Don't forget to tune in at 1 PM ET to catch @enckj @cmuffett1 @Regularisms @Science4Justice and @DSCEJ  testimony o… https://t.co/PR7lv6qZSx
RT @Dynamic_Planet: Highly protected marine areas have proved to be the most effective way to conserve and restore marine life, explains @E…
RT @OceaniMPActSA: You can also check out and download this informative MPA guide https://t.co/nSJqknryqL

#MPAGuide https://t.co/IoEEQNMgkX
The lovely music in this is by co

In [3]:
#public_tweets[0]

In [4]:
status = public_tweets[0]

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Tue Jul 07 14:00:21 +0000 2020",
    "entities": {
        "hashtags": [
            {
                "indices": [
                    0,
                    12
                ],
                "text": "INFOGRAPHIC"
            }
        ],
        "symbols": [],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1280501914740895745",
                "indices": [
                    106,
                    129
                ],
                "url": "https://t.co/6qkHVV6Bvr"
            }
        ],
        "user_mentions": []
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 1280501914740895745,
    "id_str": "1280501914740895745",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_i

In [3]:
class TweetMiner(object):

    result_limit    =   200    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        '0klZ6A3ashptZfBZpEdxIZtUn',
        'consumer_secret':     'RzDtafJJX9fO9zjykg15iMNghSMd1furq41Rpn5vfJc3M5OaQw',
        'access_token_key':    '781767392607088640-BSKrUyweBAnmAbdg1EgmxVh09zmI46y',
        'access_token_secret': 'Sg8wzQeocs7iQCZOI8QUv3bs5wOZo20MOXSgGvdwlIfiS'
    }

    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 200):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="oceana",
                         mine_rewteets=False,
                         max_pages=1):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=False
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=False)
                
            for item in statuses:
                                
                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source,
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data
    
# miner=TweetMiner(result_limit=200)
# mined_tweets = miner.mine_user_tweets(user='oceana', max_pages=25)
# mined_tweets_df= pd.DataFrame(mined_tweets)
# mined_tweets_df.to_csv('../data/oceana.csv')

In [6]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='4ocean', max_pages=25)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/4ocean.csv')

In [7]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticBank', max_pages=30)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticBank.csv')

In [8]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='TheOceanCleanup', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/TheOceanCleanup.csv')

In [9]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticBankIND', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticBankIND.csv')

In [10]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='sascampaigns', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/sascampaigns.csv')

In [11]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='Plastics4change', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/Plastics4change.csv')

In [12]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticPollutes', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticPollutes.csv')

In [13]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='aplastic_planet', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/aplastic_planet.csv')

In [14]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticOceans', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticOceans.csv')

In [15]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='5gyres', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/5gyres.csv')

In [16]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='Oceanwarrior', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/Oceanwarrior.csv')

In [17]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='plasticsoupfoun', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/plasticsoupfoun.csv')

In [18]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='NewPlasticsEcon', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/NewPlasticsEcon.csv')

In [19]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='LifeWoutPlastic', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/LifeWoutPlastic.csv')

In [20]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticFreeJuly', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticFreeJuly.csv')

In [21]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='RethinkPlastic', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/RethinkPlastic.csv')

In [22]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='JackieSavitz', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/JackieSavitz.csv')

In [23]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='bigbluecleanup', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/bigbluecleanup.csv')

In [24]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='DebrisTracker', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/DebrisTracker.csv')

In [25]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='ThePlasticWhale', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/ThePlasticWhale.csv')

In [26]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='plasticpdise', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/plasticpdise.csv')

In [27]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='TBPInnovate', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/TBPInnovate.csv')

In [28]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='merijntinga', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/merijntinga.csv')

In [29]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticsBeyond', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticsBeyond.csv')

In [30]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='OurOcean', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/OurOcean.csv')

In [31]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='oceanfdn', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/oceanfdn.csv')

In [32]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='oceanunite', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/oceanunite.csv')

In [33]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='globalalert', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/globalalert.csv')

In [34]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='PlasticOceansUS', max_pages=17)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/PlasticOceansUS.csv')

In [36]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='SylviaEarle', max_pages=20)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/SylviaEarle.csv')

In [37]:
miner=TweetMiner(result_limit=200)
mined_tweets = miner.mine_user_tweets(user='MissionBlue', max_pages=20)
mined_tweets_df= pd.DataFrame(mined_tweets)
mined_tweets_df
mined_tweets_df.to_csv('../data/MissionBlue.csv')